# Group Assignment - Natural Language Processing
##### Dennis Depoi, Anne Szyprons, Leonie Schulz, Melanie Ullrich

## Pipeline

1. Einlesen der Daten
    - alle xml-Dateien zusammenführen
    - Beschränkung auf eine Wahlperiode 
2. Datentransformation
    - Datenbereinigung (Entfernen der überflüssigen Zeichen)
    - Bereinigung der Spalte "TEXT"
    - 3 Ansätze:
        - Personenerkennung mit NER mit Spacy und nltk hat nicht funktioniert 
        - Extrahierung der Teilnehmer der Sitzungen anhand des Namensverzeichnis
        - Verwendung der Stammdatendatei (xml)
        - manuelle Liste der Abgeordneten in der Wahlperiode
    - Erstellung eines Subdataframes (Name des Vortragenden, Unterbrechungen, etc (Präsi))
3. Datenverarbeitung
    - Case Conversion
    - Tokenisierung
    - Entfernen von Stopwörtern
    - Lemmatisierung
    
    
4. Anwendung von Methoden: 
- Anwendung von Statistik: Häufigkeitsberechnung (Wie oft wer unterbrochen?)
- bag of words - tf-idf Maß - was sind häufige Wörter?
- Clustering? Einteilung in Klassen nach Wörter (neutral, Zustimmung, Ablehung)


Erstellung TF-IDF-Matrix (sklearns TfidfVectorizer)
Clustering? Topic Modelling?



## To Do
- Bereinigung der Textzeilen
- if-Schleife erweitern für mega Daten (Teilnehmer)

# 1. Einlesen der Daten

In [1]:
#settings, imports & downloads
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
#! python -m spacy download de_core_news_md

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ulli\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#settings
pd.options.display.max_colwidth = 200

In [3]:
# read xml files
folder_path = "data"
files_in_folder = os.listdir(folder_path)

result = []
for file in files_in_folder:
    soup = BeautifulSoup(open(folder_path + "/" + file, "r", encoding="utf-8"), "xml")
    d = {}
    for tag in soup.find_all(recursive=True):
        d[tag.name] = tag.get_text(strip=True)
        result.append(d)
        
#write in Dataframe
df = pd.DataFrame(result)
df = df.drop_duplicates()
df = df.drop(columns = "DOKUMENT") #delete redundant column
#df.to_csv("data_protokoll.csv") #write to csv file
df = df[:5]
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT
0,17,PLENARPROTOKOLL,17/1,27.10.2009,Plenarprotokoll vom 27.10.2009,"Plenarprotokoll 17/1\nDeutscher Bundestag\nStenografischer Bericht\n\n1. Sitzung\n\nBerlin, Dienstag, den 27. Oktober 2009\n\nI n h a l t :\nAbweichung von den Richtlinien für die Fra-\ngestunde, ..."
7,17,PLENARPROTOKOLL,17/2,28.10.2009,Plenarprotokoll vom 28.10.2009,"Plenarprotokoll 17/2\nDeutscher Bundestag\nStenografischer Bericht\n\n2. Sitzung\n\nBerlin, Mittwoch, den 28. Oktober 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nWahl der Bundeskanzlerin . . ..."
14,17,PLENARPROTOKOLL,17/4,11.11.2009,Plenarprotokoll vom 11.11.2009,"Plenarprotokoll 17/4\nDeutscher Bundestag\nStenografischer Bericht\n\n4. Sitzung\n\nBerlin, Mittwoch, den 11. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nFortsetzung der Aussprache zu..."
21,17,PLENARPROTOKOLL,17/5,12.11.2009,Plenarprotokoll vom 12.11.2009,"Plenarprotokoll 17/5\nDeutscher Bundestag\nStenografischer Bericht\n\n5. Sitzung\n\nBerlin, Donnerstag, den 12. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nFortsetzung der Aussprache ..."
28,17,PLENARPROTOKOLL,17/6,25.11.2009,Plenarprotokoll vom 25.11.2009,"Plenarprotokoll 17/6\nDeutscher Bundestag\nStenografischer Bericht\n\n6. Sitzung\n\nBerlin, Mittwoch, den 25. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nBefragung der Bundesregierung..."


In [ ]:
'''
import xml.etree.ElementTree as et 

xtree = et.parse("MDB_STAMMDATEN.xml")
xroot = xtree.getroot()

df_cols = []
rows = []

for child in xroot:
    df_cols_ = df_cols.append(child.tag)
    rows_ = rows.append(child.text)

out_df = pd.DataFrame(rows, columns = df_cols)'''

In [ ]:
'''# read xml files
result = []
soup = BeautifulSoup(open("MDB_STAMMDATEN.xml", "r", encoding="utf-8"), "xml")
d = {}
for tag in soup.find_all(recursive=True):
    d[tag.name] = tag.get_text(strip=True)
    result.append(d)
        
#write in Dataframe
stammdaten = pd.DataFrame(result)'''

In [ ]:
'''stammdaten = stammdaten[:10]
stammdaten = stammdaten.drop_duplicates()
stammdaten = stammdaten["WAHLPERIODE" == "17"]'''

In [4]:
#read csv file of the names
names = pd.read_csv("DIS25_liste_gender.csv",delimiter=";")

In [5]:
names

,Vorname_Nachname,Nachname,Vorname,Geschlecht,Fraktion
0,Jens Ackermann,Ackermann,Jens,m,FDP
1,Christian Ahrendt,Ahrendt,Christian,m,FDP
2,Ilse Aigner,Aigner,Ilse,f,CDU/CSU
3,Jan van Aken,van Aken,Jan,m,Die Linke
4,Agnes Alpers,Alpers,Agnes,f,Die Linke
...,...,...,...,...,...
647,Sabine Zimmermann,Zimmermann,Sabine,f,Die Linke
648,Wolfgang Zöller,Zöller,Wolfgang,m,CDU/CSU
649,Manfred Helmut Zöllmer,Zöllmer,Manfred Helmut,m,SPD
650,Willi Zylajew,Zylajew,Willi,m,CDU/CSU


In [6]:
names = names["Vorname_Nachname"].tolist()

# 2. Datentransformation

In [ ]:
'''
#extract the participants in columns "TEXT"
pattern_names = "Namensverzeichnis"
participants = []
for row in df["TEXT"]:
    if str(row).find(pattern_names):
        rowstr = str(row).split(pattern_names)
        participants_ = participants.append(rowstr[1].split("\n"))

#print(participants)
df["TEILNEHMER"] = participants '''

In [7]:
#split column "TEXT" and add only the important text into new column and the participants
conversation = []
pattern_start = "Beginn:"
pattern_end = "Schluss:"

for row in df["TEXT"]:
    rowstr = str(row).split(pattern_start)
    conversation.append(rowstr[1])

df["TEXT_SPLIT"] = conversation

In [8]:
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT,TEXT_SPLIT
0,17,PLENARPROTOKOLL,17/1,27.10.2009,Plenarprotokoll vom 27.10.2009,"Plenarprotokoll 17/1\nDeutscher Bundestag\nStenografischer Bericht\n\n1. Sitzung\n\nBerlin, Dienstag, den 27. Oktober 2009\n\nI n h a l t :\nAbweichung von den Richtlinien für die Fra-\ngestunde, ...","11.00 Uhr\nAlterspräsident Dr. Heinz Riesenhuber:\nGuten Morgen, meine sehr verehrten Damen und Her-\n\nren! Liebe Kolleginnen und Kollegen, ich begrüße Sie\nzur konstituierenden Sitzung des 17. ..."
7,17,PLENARPROTOKOLL,17/2,28.10.2009,Plenarprotokoll vom 28.10.2009,"Plenarprotokoll 17/2\nDeutscher Bundestag\nStenografischer Bericht\n\n2. Sitzung\n\nBerlin, Mittwoch, den 28. Oktober 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nWahl der Bundeskanzlerin . . ...","10.00 Uhr\nPräsident Dr. Norbert Lammert:\nNehmen Sie bitte Platz. Die Sitzung ist eröffnet.\n\nGuten Morgen, liebe Kolleginnen und Kollegen, ich\nbegrüße Sie alle herzlich zur zweiten Sitzung de..."
14,17,PLENARPROTOKOLL,17/4,11.11.2009,Plenarprotokoll vom 11.11.2009,"Plenarprotokoll 17/4\nDeutscher Bundestag\nStenografischer Bericht\n\n4. Sitzung\n\nBerlin, Mittwoch, den 11. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nFortsetzung der Aussprache zu...","9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet.\n\nGuten Morgen, liebe Kolleginnen und Kollegen! Ich\nbegrüße Sie herzlich zur Fortsetzung der Aussprache zur\nRegierungserklär..."
21,17,PLENARPROTOKOLL,17/5,12.11.2009,Plenarprotokoll vom 12.11.2009,"Plenarprotokoll 17/5\nDeutscher Bundestag\nStenografischer Bericht\n\n5. Sitzung\n\nBerlin, Donnerstag, den 12. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nFortsetzung der Aussprache ...","9.00 Uhr\nVizepräsidentin Gerda Hasselfeldt:\nIch eröffne die Sitzung.\n\nIch begrüße Sie sehr herzlich zur Fortsetzung unserer\nheutigen Beratungen, bei denen es um die weitere Aus-\nsprache zur..."
28,17,PLENARPROTOKOLL,17/6,25.11.2009,Plenarprotokoll vom 25.11.2009,"Plenarprotokoll 17/6\nDeutscher Bundestag\nStenografischer Bericht\n\n6. Sitzung\n\nBerlin, Mittwoch, den 25. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nBefragung der Bundesregierung...","13.00 Uhr\nVizepräsidentin Petra Pau:\nGuten Tag, liebe Kolleginnen und Kollegen! Die Sit-\n\nzung ist eröffnet.\n\nIch rufe den Tagesordnungspunkt 1 auf:\n\nBefragung der Bundesregierung\n\nDie ..."


In [ ]:
'''#split column "TEXT" and add only the important text into new column and the participants
conversation = []
participants = []
pattern_start = "Beginn:"
pattern_end = "Schluss:"
pattern_names = "Namensverzeichnis"

for row in df["TEXT"]:
    rowstr_start = str(row).split(pattern_start)
    rowstr_start1 = rowstr_start[1]
    rowstr_names = rowstr_start1.split(pattern_names)
    conversation.append(rowstr_names[0])
    participants.append(rowstr_names[1])

df["TEXT_SPLIT"] = conversation
df["TEILNEHMER"] = participants'''

In [9]:
#clean data
df["TEXT_SPLIT"] = df.apply(lambda row: re.sub("(\n)|(\r)|(\s+)", " ", row["TEXT_SPLIT"]), axis=1) #delete returns in data
df["TEXT_SPLIT"] = df.apply(lambda row: re.sub("-\s", "", row["TEXT_SPLIT"]), axis=1) #delete separate words with hyphen

In [10]:
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT,TEXT_SPLIT
0,17,PLENARPROTOKOLL,17/1,27.10.2009,Plenarprotokoll vom 27.10.2009,"Plenarprotokoll 17/1\nDeutscher Bundestag\nStenografischer Bericht\n\n1. Sitzung\n\nBerlin, Dienstag, den 27. Oktober 2009\n\nI n h a l t :\nAbweichung von den Richtlinien für die Fra-\ngestunde, ...","11.00 Uhr Alterspräsident Dr. Heinz Riesenhuber: Guten Morgen, meine sehr verehrten Damen und Her ren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutsch..."
7,17,PLENARPROTOKOLL,17/2,28.10.2009,Plenarprotokoll vom 28.10.2009,"Plenarprotokoll 17/2\nDeutscher Bundestag\nStenografischer Bericht\n\n2. Sitzung\n\nBerlin, Mittwoch, den 28. Oktober 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nWahl der Bundeskanzlerin . . ...","10.00 Uhr Präsident Dr. Norbert Lammert: Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen, ich begrüße Sie alle herzlich zur zweiten Sitzung des Deu..."
14,17,PLENARPROTOKOLL,17/4,11.11.2009,Plenarprotokoll vom 11.11.2009,"Plenarprotokoll 17/4\nDeutscher Bundestag\nStenografischer Bericht\n\n4. Sitzung\n\nBerlin, Mittwoch, den 11. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nFortsetzung der Aussprache zu...","9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie herzlich zur Fortsetzung der Aussprache zur Regierungserklärung de..."
21,17,PLENARPROTOKOLL,17/5,12.11.2009,Plenarprotokoll vom 12.11.2009,"Plenarprotokoll 17/5\nDeutscher Bundestag\nStenografischer Bericht\n\n5. Sitzung\n\nBerlin, Donnerstag, den 12. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nFortsetzung der Aussprache ...","9.00 Uhr Vizepräsidentin Gerda Hasselfeldt: Ich eröffne die Sitzung. Ich begrüße Sie sehr herzlich zur Fortsetzung unserer heutigen Beratungen, bei denen es um die weitere Aussprache zur Regieru..."
28,17,PLENARPROTOKOLL,17/6,25.11.2009,Plenarprotokoll vom 25.11.2009,"Plenarprotokoll 17/6\nDeutscher Bundestag\nStenografischer Bericht\n\n6. Sitzung\n\nBerlin, Mittwoch, den 25. November 2009\n\nI n h a l t :\nTagesordnungspunkt 1:\n\nBefragung der Bundesregierung...","13.00 Uhr Vizepräsidentin Petra Pau: Guten Tag, liebe Kolleginnen und Kollegen! Die Sit zung ist eröffnet. Ich rufe den Tagesordnungspunkt 1 auf: Befragung der Bundesregierung Die Bundesregier..."


## Neues SubDataframe

In [ ]:
'''row = "11.00 Uhr Alterspräsident Dr. Heinz Riesenhuber: Guten Morgen, meine sehr verehrten Damen und Her ren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutschen Bundestags.  Parlamentarischer Brauch ist es – das entspricht unserer Geschäftsordnung; ich kann die Paragrafen zitieren –, dass der Älteste die erste Sitzung des Bundestags eröffnet. Ich bin am Sonntag, dem 1. Dezember 1935, geboren. Wenn jemand von den Kollegen im Saal älter ist als ich, dann spreche er jetzt oder er schweige für immer.  (Heiterkeit und Beifall) "
x = "Heinz Riesenhuber"

start = row.index(str(x) + ":")
end = row.index(')',start+1)
substring = row[start-1:end+1]
    

print(f"Start: {start}, End: {end}")
print(substring)'''

In [11]:
# write the components of speaker, speech and interruption
Sub = []

#while-Schleife drum rum schreiben?

for row in df["TEXT_SPLIT"]:
    for i in names:
        x = str(i) + ":"
        if x in row:
            start = str(row).index(x)
            end = row.index(')',start+1)
            substring = row[start-1:end+1]
            Sub.append(substring)
        else:
            continue

In [14]:
#split the components and write into new dataframe
data = pd.DataFrame()
Redebeitrag = []
Person = []
Unterbrechung = []


for x in Sub:
    string = re.split(': | \\(|\\)', str(x))
    Person.append(string[0])
    Redebeitrag.append(string[1])
    Unterbrechung.append(string[2])

data["Redeinhalt"] = Redebeitrag
data["Vortragende_Person"] = Person
data["Unterbrechung"] = Unterbrechung

In [15]:
data

,Redeinhalt,Vortragende_Person,Unterbrechung
0,"496 Jastimmen,",Gerda Hasselfeldt,Beifall im ganzen Hause
1,Sehr geehrter Herr Bundespräsident! Verehrte Gäste! Liebe Kolleginnen und Kollegen! Die erneute Wahl zum Präsidenten des Deutschen Bundestages ist eine hohe Auszeichnung und eine große Verpflicht...,Norbert Lammert,"Heiterkeit und Beifall bei der CDU/CSU, der FDP sowie bei Abgeordneten des BÜNDNISSES 90/DIE GRÜNEN"
2,"Guten Morgen, meine sehr verehrten Damen und Her ren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutschen Bundestags. Parlamentarischer Brauch ist es – ...",Heinz Riesenhuber,Heiterkeit und Beifall
3,"Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen, ich begrüße Sie alle herzlich zur zweiten Sitzung des Deutschen Bundestages in der neuen Legislatur...",Norbert Lammert,Wahl der Bundeskanzlerin Dazu hat mir der Herr Bundespräsident mitgeteilt
4,Das Wort hat jetzt der Kollege Dr. Günter Krings für die CDU/CSU-Fraktion.,Katrin Göring-Eckardt,Beifall bei der CDU/CSU sowie bei Abgeordneten der FDP
5,Nächster Redner ist der Kollege Dirk Fischer für die CDU/CSU-Fraktion.,Gerda Hasselfeldt,Beifall bei der CDU/CSU sowie bei Abgeordneten der FDP
6,"Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie herzlich zur Fortsetzung der Aussprache zur Regierungserklärung der Bundeskanzlerin und rufe Tagesordnungsp...",Norbert Lammert,"Regierungserklärung der Bundeskanzlerin mit anschließender Aussprache Ich darf Sie daran erinnern, dass wir gestern für die heutige Aussprache zehneinviertel Stunden beschlossen haben – mit eine..."
7,Für die Fraktion BÜNDNIS 90/DIE GRÜNEN spricht nun die Kollegin Krista Sager. Krista Sager,Petra Pau,BÜNDNIS 90/DIE GRÜNEN
8,"Ich habe vor zwei Jahren bei der Beerdigung der Transrapid-Pläne schon gesagt, dass hier das letzte Wort noch nicht gesprochen ist …",Peter Ramsauer,Heiterkeit bei der SPD
9,Das Wort hat der Kollege Max Straubinger von der CDU/CSU-Fraktion.,Hermann Otto Solms,Beifall bei Abgeordneten der CDU/CSU – Dr. Heinrich L. Kolb [FDP]


# 3. Datenverarbeitung

In [17]:
#case conversion & tokenizer
tokenizer = RegexpTokenizer(r'\w+')
data["TOKENS"] = data.apply (lambda row: tokenizer.tokenize(str(row["Unterbrechung"].lower())), axis=1)

In [18]:
#stopwords removal
def remove_stopwords_from_list(list_in, language):
    return [a for a in list_in if a not in nltk.corpus.stopwords.words(language)]

In [19]:
data["TOKENS"] = data.apply(lambda row: remove_stopwords_from_list(row["TOKENS"], "german"), axis=1)

In [ ]:
'''##doesn`t work
nlp = spacy.load("de_core_news_md")
nlp.max_length = 2000000

words = {}
for row in df["TOKENS"]:
    doc = nlp(str(row))
    for token in doc.ents:
        words = {token.text: token.label_}
        
nlp = spacy.load("de_core_news_md")
nlp.max_length = 2000000
liste = []
doc = nlp(str(df["TEXT_SPLIT"]))
liste.append([(X.text, X.label_) for X in doc.ents])
df["NER"] = liste'''

In [ ]:
'''#doesn't work
import nltk

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
nltk.download("punkt")
nltk.download('maxent_ne_chunker')
nltk.download('words')

text =
Alterspräsident Dr. Heinz Riesenhuber
und es wird weltweit geteilt. Die weltweite Gemein-
schaft, die Wissensgesellschaft bietet jedem offenen Zu-
griff auf alles, wenn er alles, was er hat, mit einbringt. Es
ist eine Gesellschaft, die aus Wissen lebt, die mit Wissen
wächst, die mit Wissen verantwortlich umzugehen ver-
steht, in der Wissen überall und jederzeit für jeden zu-
gänglich ist.

Es ist die Welt der Quanten, die wir langsam verste-
hen lernen  von der Vision des Quantencomputers bis
zur Nanotechnologie, wo im Allerkleinsten die Materie
andere Eigenschaften hat. Es ist die Welt der Gene, die
neue Möglichkeiten schafft. Nur ein Viertel der Krank-
heiten, die wir kennen, können wir an der Wurzel fassen.
Gentechnologie eröffnet die Chance, dass wir Krankhei-
ten heilen, dass wir menschliches Leid lindern, dass wir
gleichzeitig aus Wissen neue Märkte, neue Arbeitsplätze
und wirtschaftliches Wachstum schaffen.

Es ist die Welt der Computer, die schon jetzt unseren
Alltag beherrscht. Die Leistung der Chips verdoppelt
sich alle 18 Monate, und es halbiert sich ihr Preis. Da-
raus entstehen Internet und Unterhaltungselektronik.
Daraus entsteht eine Fülle von Möglichkeiten in Chemie
und im Maschinenbau. Daraus entsteht eine Fülle von
Möglichkeiten im Automobilbau. Daraus entstehen lau-
ter Arbeitsplätze, die für unsere Zukunft wichtig sind, in
einem Bereich, wo Deutschland immer stark war.
Deutschland war selten in den Spitzentechnologien
vorne, aber als Meister der Systeme wurden aus den ein-
zelnen Techniken neue Produkte geschaffen.

Der Chip ist Silizium, und Silizium ist Sand  und
Sand ist reichlich vorhanden. Und dazu braucht man
noch Intelligenz. Intelligenz, so sagt man uns, ist belie-
big vorhanden. In der Praxis findet man gelegentlich
Knappheiten.

(Heiterkeit bei Abgeordneten der CDU/CSU
und der FDP)

Aber sie wächst nach; es bleibt Hoffnung. Insofern ha-
ben wir hier die Chance eines Wachstums des guten Ge-
wissens, das die Wirklichkeit ändert und Zukunft für alle
schafft.

Dass diese Welt gelingt, hängt davon ab, ob wir die
Menschen haben, die sie gestalten, die Freude daran ha-
ben und die ihre Chance darin sehen. Wir werden uns
über vieles streiten, und wir haben uns über vieles ge-
stritten  manchmal mit intensiver Herzlichkeit. Aber
wir sind uns doch wohl weitestgehend darin einig, dass
Bildung und Forschung in diesen Jahren hohe Priorität
haben. Dafür braucht es nicht allein Geld  das auch ,
sondern auch Konzepte und unsere Bereitschaft, denen,
die hier gestalten, die Freude daran nicht zu nehmen.
Wir haben in diesen Jahren in einer Welt im Wandel die
Schulen umzubauen und aufzubauen. Wir haben gute
Schulen, und seit PISA sind sie noch besser geworden;
aber wir wissen, dass wir noch mehr erreichen können.

In der Föderalismusreform haben wir den Ländern die
Zuständigkeiten für die Bildung weitgehend zugeschrie-
ben. Sie wetteifern um das beste Schulsystem und um
die besten Chancen. Den Universitäten haben wir durch
einen Wettbewerb um Exzellenz, der über zehn Jahre an-
gelegt worden ist, den Ehrgeiz vermittelt, die Besten zu
sein und von jedem anderen zu lernen. Der Forschung
haben wir langfristig steigende Mittel zugesagt.

Was wir hier machen, ist eine Investition dafür, dass
wir stärker sind, wenn wir aus der Krise herauskommen.
Wir brauchen diese Stärke. Es gibt Aufgaben über das
Tagesgeschäft hinaus, die langfristig sind und in unter-
schiedlichen Bereichen angegangen werden müssen. Die
Menschen werden älter. Das ist ein Erfolg der Medizin.
Das ist ein Erfolg der Arbeitsbedingungen, die besser
geworden sind. Das ist ein Erfolg der Sozialsysteme.
Das ist also eine erfreuliche Sache und kein Problem.
Das Problem liegt darin, dass wir nicht rechtzeitig so
viele Kinder gezeugt haben, wie wir es hätten tun sollen.

(Heiterkeit)

Was hier und jetzt anzugehen ist,

(Renate Künast [BÜNDNIS 90/DIE GRÜ-
NEN]: Das wollen Sie jetzt angehen?  Heiter-
keit)

ist, dass wir in dieser Gesellschaft, die altert, die Kreati-
vität bewahren und das dritte Lebensalter nicht als ein
Zu-Ende-Leben des Lebens verstehen, sondern als neuen
Raum der Gestaltung mit einer Freiheit, die man über
das ganze frühere Leben nicht hatte. Man kann wählen;
das gilt sowohl für die Freizeit, das Ehrenamt und, wenn
man es will und kann, die Arbeit. Dies so zu organisie-
ren, dass die Menschen diese Chance ergreifen, dass sie
mitgestalten und aktiv dabei sind, ist eine unserer großen
Aufgaben.


nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
        name = ''
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
        print ('Type: ', nltk_result.label(), 'Name: ', name)'''

In [20]:
#pos tagging
words = []
for row in data["TOKENS"]:
    tagged_tokens = nltk.pos_tag(row)
    words_ = words.append(tagged_tokens)

data["POS"] = words

In [21]:
#lemmatizer
lemmatizer = WordNetLemmatizer()
data["LEMMATA"] = data.apply(lambda row: [lemmatizer.lemmatize(word) for word in row["TOKENS"]], axis=1)

In [22]:
data

,Redeinhalt,Vortragende_Person,Unterbrechung,TOKENS,POS,LEMMATA
0,"496 Jastimmen,",Gerda Hasselfeldt,Beifall im ganzen Hause,"[beifall, ganzen, hause]","[(beifall, DT), (ganzen, NN), (hause, NN)]","[beifall, ganzen, hause]"
1,Sehr geehrter Herr Bundespräsident! Verehrte Gäste! Liebe Kolleginnen und Kollegen! Die erneute Wahl zum Präsidenten des Deutschen Bundestages ist eine hohe Auszeichnung und eine große Verpflicht...,Norbert Lammert,"Heiterkeit und Beifall bei der CDU/CSU, der FDP sowie bei Abgeordneten des BÜNDNISSES 90/DIE GRÜNEN","[heiterkeit, beifall, cdu, csu, fdp, sowie, abgeordneten, bündnisses, 90, grünen]","[(heiterkeit, NN), (beifall, NN), (cdu, NN), (csu, NN), (fdp, NN), (sowie, NN), (abgeordneten, VBP), (bündnisses, VBZ), (90, CD), (grünen, NN)]","[heiterkeit, beifall, cdu, csu, fdp, sowie, abgeordneten, bündnisses, 90, grünen]"
2,"Guten Morgen, meine sehr verehrten Damen und Her ren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutschen Bundestags. Parlamentarischer Brauch ist es – ...",Heinz Riesenhuber,Heiterkeit und Beifall,"[heiterkeit, beifall]","[(heiterkeit, NN), (beifall, NN)]","[heiterkeit, beifall]"
3,"Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen, ich begrüße Sie alle herzlich zur zweiten Sitzung des Deutschen Bundestages in der neuen Legislatur...",Norbert Lammert,Wahl der Bundeskanzlerin Dazu hat mir der Herr Bundespräsident mitgeteilt,"[wahl, bundeskanzlerin, herr, bundespräsident, mitgeteilt]","[(wahl, NN), (bundeskanzlerin, NN), (herr, NN), (bundespräsident, NN), (mitgeteilt, NN)]","[wahl, bundeskanzlerin, herr, bundespräsident, mitgeteilt]"
4,Das Wort hat jetzt der Kollege Dr. Günter Krings für die CDU/CSU-Fraktion.,Katrin Göring-Eckardt,Beifall bei der CDU/CSU sowie bei Abgeordneten der FDP,"[beifall, cdu, csu, sowie, abgeordneten, fdp]","[(beifall, NN), (cdu, NN), (csu, NN), (sowie, NN), (abgeordneten, VBP), (fdp, NN)]","[beifall, cdu, csu, sowie, abgeordneten, fdp]"
5,Nächster Redner ist der Kollege Dirk Fischer für die CDU/CSU-Fraktion.,Gerda Hasselfeldt,Beifall bei der CDU/CSU sowie bei Abgeordneten der FDP,"[beifall, cdu, csu, sowie, abgeordneten, fdp]","[(beifall, NN), (cdu, NN), (csu, NN), (sowie, NN), (abgeordneten, VBP), (fdp, NN)]","[beifall, cdu, csu, sowie, abgeordneten, fdp]"
6,"Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie herzlich zur Fortsetzung der Aussprache zur Regierungserklärung der Bundeskanzlerin und rufe Tagesordnungsp...",Norbert Lammert,"Regierungserklärung der Bundeskanzlerin mit anschließender Aussprache Ich darf Sie daran erinnern, dass wir gestern für die heutige Aussprache zehneinviertel Stunden beschlossen haben – mit eine...","[regierungserklärung, bundeskanzlerin, anschließender, aussprache, darf, daran, erinnern, gestern, heutige, aussprache, zehneinviertel, stunden, beschlossen, vereinbarung, reihenfolge, fachbereich...","[(regierungserklärung, NN), (bundeskanzlerin, NN), (anschließender, NN), (aussprache, NN), (darf, NN), (daran, NN), (erinnern, JJ), (gestern, JJ), (heutige, NN), (aussprache, NN), (zehneinviertel,...","[regierungserklärung, bundeskanzlerin, anschließender, aussprache, darf, daran, erinnern, gestern, heutige, aussprache, zehneinviertel, stunden, beschlossen, vereinbarung, reihenfolge, fachbereich..."
7,Für die Fraktion BÜNDNIS 90/DIE GRÜNEN spricht nun die Kollegin Krista Sager. Krista Sager,Petra Pau,BÜNDNIS 90/DIE GRÜNEN,"[bündnis, 90, grünen]","[(bündnis, NN), (90, CD), (grünen, NN)]","[bündnis, 90, grünen]"
8,"Ich habe vor zwei Jahren bei der Beerdigung der Transrapid-Pläne schon gesagt, dass hier das letzte Wort noch nicht gesprochen ist …",Peter Ramsauer,Heiterkeit bei der SPD,"[heiterkeit, spd]","[(heiterkeit, NN), (spd, NN)]","[heiterkeit, spd]"
9,Das Wort hat der Kollege Max Straubinger von der CDU/CSU-Fraktion.,Hermann Otto Solms,Beifall bei Abgeordneten der CDU/CSU – Dr. Heinrich L. Kolb [FDP],"[beif

In [ ]:
'''#create new Dataframe and copy some old columns
data = df[["NR","DATUM","TEXT_SPLIT"]].copy()'''